In [3]:
import os
import sys
os.chdir('../')

In [4]:
pwd

'c:\\Users\\Viral Sherathiya\\Desktop\\NLP Data\\Text_Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarization.constants import *
from textSummarization.utils.common import *
from textSummarization import logger,CustomException



class ConfigurationManager:

    def __init__(
            self,
            config_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        try:
            config = self.config.data_transformation
            print(config)
            create_directories([config.root_dir])

            data_transformation_config = DataTransformationConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                tokenizer_name=config.tokenizer_name,
            )

            return data_transformation_config
        except Exception as e:
                logger.info(f"Error Occurred at {CustomException(e,sys)}")
                raise CustomException(e, sys)

In [9]:
from textSummarization import logger,CustomException
from datasets import load_dataset,load_from_disk
import os
from transformers import AutoTokenizer




class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
        
    
    
    def convert_examples_to_features(self,example_batch):
        try:
            input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )


            with self.tokenizer.as_target_tokenizer():
                target_encodings  = self.tokenizer(example_batch['summary'],max_length=1024,truncation=True )

            return {
                'input_ids' : input_encodings['input_ids'],
                'attention_mask': input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']
            }
        except Exception as e:
                    logger.info(f"Error Occurred at {CustomException(e,sys)}")
                    raise CustomException(e, sys)
                
    def convert(self):
        try:
            dataset_samsum = load_from_disk(self.config.data_path)
            dataset_samsum_pt= dataset_samsum.map(self.convert_examples_to_features,batched=True)
            dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
        except Exception as e:
                    logger.info(f"Error Occurred at {CustomException(e,sys)}")
                    raise CustomException(e, sys)

In [10]:
try:
    config =  ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    logger.info(f"Error Occured at {CustomException(e,sys)}")
    raise CustomException(e,sys)

[2023-08-13 17:21:11,232] || common.py || 37 || TextSummarizerLogger || read_yaml() || 37 || INFO || yaml file : config\config.yaml loaded successfully
[2023-08-13 17:21:11,235] || common.py || 37 || TextSummarizerLogger || read_yaml() || 37 || INFO || yaml file : config\params.yaml loaded successfully
[2023-08-13 17:21:11,236] || common.py || 69 || TextSummarizerLogger || create_directories() || 69 || INFO || Directory artifacts created successfully
{'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}
[2023-08-13 17:21:11,238] || common.py || 69 || TextSummarizerLogger || create_directories() || 69 || INFO || Directory artifacts/data_transformation created successfully


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

c:\Users\Viral Sherathiya\Desktop\NLP Data\Text_Summarizer\venv\lib\site-packages\transformers\tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14732 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/818 [00:00<?, ? examples/s]